<a href="https://colab.research.google.com/github/echuung94/Heart_Study/blob/aholley/Heart_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Predicting Heart Disease**

Import Dependencies

In [ ]:
from path import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Import Data

In [ ]:
#data = Path for data
#df = 
#df.head()

Separate the Features (X) from the Target (y)

Split our data into training and testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

Create a Logistic Regression Model

Fit (train) or model using the training data

Make predictions